# Performance optimization overview

This tutorial illustrates the performance optimizations applied to the code generated by an `Operator`. As we shall see, most optimizations are automatically applied as they're known to systematically improve performance. Others, whose impact varies across different `Operator`s, are instead to be enabled through specific options.

An Operator has several preset **optimization levels**; the fundamental ones are `noop` and `advanced`. With `noop`, no performance optimizations are introduced by the compiler. With `advanced`, several flop-reducing and data locality _optimization passes_ are performed. Examples of flop-reducing optimization passes are common sub-expressions elimination and factorization; examples of data locality optimization passes are loop fusion and cache blocking. SIMD vectorization, via compiler auto-vectorization, is enforced through OpenMP pragmas.

An optimization pass may provide knobs, or **options**, for fine-grained tuning. As explained in the next sections, some of these options are given at compile-time, others at run-time.

**\*\* Remark \*\***

Parallelism -- both shared-memory (e.g., OpenMP) and distributed-memory (MPI) -- is _by default disabled_ and is _not_ controlled via the optimization level. In this tutorial we will also show how to enable OpenMP parallelism (you'll see it's trivial!). A mini-guide about parallelism in Devito and related aspects is also available [here](https://github.com/devitocodes/devito/tree/master/benchmarks/user#a-step-back-configuring-your-machine-for-reliable-benchmarking). 

**\*\*\*\***

## API

The optimization level may be changed in various ways:

* globally, through the `DEVITO_OPT` environment variable. For example, to disable all optimizations on all `Operator`s, one should run with

```
DEVITO_OPT=noop python ...
```

* programmatically, adding the following lines to a program

```
from devito import configuration
configuration['opt'] = 'noop'
```

* locally, as an `Operator` argument

```
Operator(..., opt='noop')
```

Local takes precedence over programmatic, and programmatic takes precedence over global.

The optimization options, instead, may only be changed locally. The syntax to specify an option is

```
Operator(..., opt=('advanced', {<optimization options>})
```

A concrete example (you can ignore the meaning for now) is

```
Operator(..., opt=('advanced', {'blocklevels': 2})
```

That is, options are to be specified _together with_ the optimization level.

## Default values

By default, all `Operator`s are run with the optimization level set to `advanced` and with all options disabled. So this

```
Operator(Eq(...))
```

is equivalent to

```
Operator(Eq(...), opt='advanced')
```

and obviously also to

```
Operator(Eq(...), opt=('advanced', {}))
```

In virtually all scenarios, regardless of application and underlying architecture, this ensures very good performance -- but not necessarily the very best.

## Utilities

The following functions will be used throughout the notebook for various purposes.

In [6]:
#TODO: George -- I guess we could (and should!) move this under examples/performance/utils.py ... 
def _unidiff_output(expected, actual):
    """
    Helper function. Returns a string containing the unified diff of two multiline strings.
    """
    import difflib
    expected=expected.splitlines(1)
    actual=actual.splitlines(1)

    diff=difflib.unified_diff(expected, actual)

    return ''.join(diff)

## Running example

Throughout the notebook we will generate `Operator`s for the following time-marching `Eq`.

In [1]:
from sympy import sin
from devito import Eq, Grid, Operator, Function, TimeFunction

grid = Grid(shape=(80, 80, 80))

f = Function(name='f', grid=grid)
u = TimeFunction(name='u', grid=grid, space_order=2)

eq = Eq(u.forward, sin(f)*u.dy.dy)

Despite its simplicity, this `Eq` is all we need to showcase the key components of the Devito optimization engine.

## OpenMP Parallelism

There are several ways to enable OpenMP parallelism. The one we use here consists of supplying an **option** to an `Operator`. The next cell illustrates the difference between two `Operator`s generated with the `noop` optimization level, but with OpenMP enabled on the latter `Operator`. 

In [2]:
# NOTE: we only need this cell for Continuous Integration, which runs with OpenMP enabled
from devito import configuration
configuration['language'] = 'C'

In [3]:
op0 = Operator(eq, opt=('noop'))
op0_omp = Operator(eq, opt=('noop', {'openmp': True}))

# print(op0)
# print(_unidiff_output(str(op0), str(op0_omp)))  # Uncomment to print out the diff only
print(op0_omp)

#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "omp.h"

struct dataobj
{
  void *restrict data;
  int * size;
  int * npsize;
  int * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
} ;


int Kernel(struct dataobj *restrict f_vec, const float h_y, struct dataobj *restrict u_vec, const int time_M, const int time_m, struct profiler * timers, const int x_M, const int x_m, const int y_M, const int y_m, const int z_M, const int z_m, const int nthreads)
{
  float (*restrict f)[f_vec->size[1]][f_vec->size[2]] __attribute__ ((aligned (64))) = (float (*)[f_vec->size[1]][f_vec->size[2]]) f_vec->data;
  float (*restrict u)[u_vec->size[1]][u_vec->size[2]][u_vec->size[3]] __attribute__ ((aligned (64))) = (float (*)[u_vec->size[1]][u_vec->size[2]][u_vec->size[3]]) u_vec->data;

  for (int time = time_m, t0 = (time)%(2), t1 = (time + 1)%(2); time <= time_M; time += 1, t0 = (time)%(2), t1 = (time + 1)%(2)

The OpenMP-ized `op0_omp` `Operator`:

 - includes the necessary header file `#include "omp.h"`
 - the `#pragma omp parallel num_threads(nthreads)` directive
 - the `#pragma omp for collapse(...) schedule(dynamic,1)` directive
 
More complex `Operator`s will have more directives, more types of directives, different iteration scheduling strategies based on heuristics and empirical tuning (e.g., `static` instead of `dynamic`), etc.

We note how the OpenMP optimization pass also introduces a new symbol, `nthreads`. This allows users to explicitly control the number of threads with which an `Operator` is run.

In [ ]:
op0_omp.apply(time_M=0)  # Picks up `nthreads` from the standard environment variable OMP_NUM_THREADS
op0_omp.apply(time_M=0, nthreads=2)  # Runs with 2 threads per parallel loop

## The `advanced` mode

As already explained, `advanced` is the default optimization level in Devito. This mode performs several compilation passes to optimize the `Operator` for computation (number of flops), working set size, and data locality.

In the next paragraphs we dissect the `advanced` mode and analyze, one by one, some of its key passes.

### Loop blocking

The next cell creates a new `Operator` that adds loop blocking to what we had in `op0_omp`. 

In [ ]:
op1_omp = Operator(eq, opt=('blocking', {'openmp': True}))
print(op1_omp)

**\*\* Remark \*\***

`'blocking'` is **not** an optimization level -- it rather identifies a very specific compilation pass. In other words, the `advanced` mode represents a set of passes, and `blocking` is one such pass.

**\*\*\*\***

The `blocking` pass creates additional loops over blocks. In this simple `Operator` there is just one loop nest, so only a pair of additional loops are created. In more complex `Operator`s, several loop nests may individually be blocked, whereas others may be left unblocked -- this is decided by the Devito compiler according to certain heuristics. The size of a block is represented by the symbols `x0_blk0_size` and `y0_blk0_size`, which are runtime parameters akin to `nthreads`. 

By default, Devito applies 2D blocking and sets the default block shape to 8x8. There are two ways to set a different block shape:

* passing an explicit value. In the example below we would run with a 24x8 block shape

```
op1_omp.apply(..., x0_blk0_size=24)
```

* letting the autotuner run for a while, looking for a better block shape. There are several autotuning modes. A short summary is available [here](https://github.com/devitocodes/devito/wiki/FAQ#devito_autotuning)

```
op1_omp.apply(..., autotune='aggressive')
```

Loop blocking also provides two optimization options:

* `blockinner={False, True}` -- to enable 3D (or any nD, n>2) blocking
* `blocklevels={int}` -- to enable hierarchical blocking, to exploit the cache hierarchy 

In the example below, we construct an `Operator` featuring six-dimensional loop blocking, where the first three loops represent outer blocks, whereas the second three loops represent inner blocks within an outer block.

In [ ]:
op1_omp_6D = Operator(eq, opt=('blocking', {'blockinner': True, 'blocklevels': 2, 'openmp': True}))
print(op1_omp_6D)

### SIMD vectorization

Devito enforces SIMD vectorization through OpenMP pragmas.

In [ ]:
op2_omp = Operator(eq, opt=('blocking', 'simd', {'openmp': True}))
# print(op2_omp)  # Uncomment to see the generated code
# print(_unidiff_output(str(op1_omp), str(op2_omp)))  # Uncomment to print out the diff only

### Code motion

The `advanced` mode performs a code motion pass. In explicit PDE solvers, this is most commonly used to lift expensive time-invariant sub-expressions out of the inner loops. The pass is however quite general in that it is not restricted to the concept of time-invariance -- any sub-expression invariant with respect to a subset of `Dimension`s is a code motion candidate. In the example below, we see that `sin(f)` gets hoisted out of the inner loops since it is determined to be an expensive invariant sub-expression. In other words, the compiler trades the redundant computation of `sin(f)` for additional storage (the `r1[...]` array).

In [8]:
op3_omp = Operator(eq, opt=('lift', {'openmp': True}))
print(op3_omp)

#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "omp.h"

struct dataobj
{
  void *restrict data;
  int * size;
  int * npsize;
  int * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
  double section1;
} ;


int Kernel(struct dataobj *restrict f_vec, const float h_y, struct dataobj *restrict u_vec, const int x_size, const int y_size, const int z_size, const int time_M, const int time_m, struct profiler * timers, const int x_M, const int x_m, const int y_M, const int y_m, const int z_M, const int z_m, const int nthreads)
{
  float (*restrict f)[f_vec->size[1]][f_vec->size[2]] __attribute__ ((aligned (64))) = (float (*)[f_vec->size[1]][f_vec->size[2]]) f_vec->data;
  float (*restrict u)[u_vec->size[1]][u_vec->size[2]][u_vec->size[3]] __attribute__ ((aligned (64))) = (float (*)[u_vec->size[1]][u_vec->size[2]][u_vec->size[3]]) u_vec->data;

  float (*r1)[y_size][z_size];
  posix_memalign((void*

### Simple flop-reducing transformations

Among the simpler flop-reducing transformations applied by the `advanved` mode we find:

* "classic" common sub-expressions elimination (CSE),
* factorization,
* optimization of powers

The cell below shows how the computation of `u` changes by incrementally applying these three passes. First of all, we observe how the symbolic spacing `h_y` gets assigned to a temporary, `r0`, due to appearing in several sub-expressions. This is the effect of CSE.

In [11]:
op4_omp = Operator(eq, opt=('cse', {'openmp': True}))
print(_unidiff_output(str(op0_omp), str(op4_omp)))

--- 
+++ 
@@ -40,7 +40,8 @@
         {
           for (int z = z_m; z <= z_M; z += 1)
           {
-            u[t1][x + 2][y + 2][z + 2] = (-(-u[t0][x + 2][y + 2][z + 2]/h_y + u[t0][x + 2][y + 3][z + 2]/h_y)/h_y + (-u[t0][x + 2][y + 3][z + 2]/h_y + u[t0][x + 2][y + 4][z + 2]/h_y)/h_y)*sin(f[x + 1][y + 1][z + 1]);
+            float r0 = 1.0/h_y;
+            u[t1][x + 2][y + 2][z + 2] = (-r0*(-r0*u[t0][x + 2][y + 2][z + 2] + r0*u[t0][x + 2][y + 3][z + 2]) + r0*(-r0*u[t0][x + 2][y + 3][z + 2] + r0*u[t0][x + 2][y + 4][z + 2]))*sin(f[x + 1][y + 1][z + 1]);
           }
         }
       }



Factorization causes the collection of `r0` in two different sub-expressions. In larger, more complex examples, the impact of factorization is typically much more pronounced.

In [13]:
op5_omp = Operator(eq, opt=('cse', 'factorize', {'openmp': True}))
print(_unidiff_output(str(op5_omp), str(op4_omp)))

--- 
+++ 
@@ -41,7 +41,7 @@
           for (int z = z_m; z <= z_M; z += 1)
           {
             float r0 = 1.0/h_y;
-            u[t1][x + 2][y + 2][z + 2] = (r0*(-r0*(-u[t0][x + 2][y + 2][z + 2]) - r0*u[t0][x + 2][y + 3][z + 2]) + r0*(r0*(-u[t0][x + 2][y + 3][z + 2]) + r0*u[t0][x + 2][y + 4][z + 2]))*sin(f[x + 1][y + 1][z + 1]);
+            u[t1][x + 2][y + 2][z + 2] = (-r0*(-r0*u[t0][x + 2][y + 2][z + 2] + r0*u[t0][x + 2][y + 3][z + 2]) + r0*(-r0*u[t0][x + 2][y + 3][z + 2] + r0*u[t0][x + 2][y + 4][z + 2]))*sin(f[x + 1][y + 1][z + 1]);
           }
         }
       }



There are no powers in this example, so the `opt-pows` pass has no effect.

In [14]:
op6_omp = Operator(eq, opt=('cse', 'factorize', 'opt-pows', {'openmp': True}))
print(_unidiff_output(str(op6_omp), str(op5_omp)))

### Cross-iteration redundancy elimination (CIRE)

...

### All optimizations together

...

In [ ]:
op = Operator(eq, openmp=True)  # `openmp=True` shortcut for `opt=('advanced', {'openmp': True})`
print(op)

There are many other things that the `advanced` mode does which are not described in this introductory tutorial. Some are visible printing `op` (e.g., the addition of denormal-numbers related macros), others are not (e.g., all of the MPI-related optimizations). This will be covered in another tutorial.

# References

- Michael E. Wolf and Monica S. Lam. 1991. A data locality optimizing algorithm. In Proceedings of the ACM SIGPLAN 1991 conference on Programming language design and implementation (PLDI ’91). Association for Computing Machinery, New York, NY, USA, 30–44. DOI:https://doi.org/10.1145/113445.113449